Pierre Navaro - [Institut de Recherche Mathématique de Rennes](https://irmar.univ-rennes1.fr) - [CNRS](http://www.cnrs.fr/)

In [30]:
%matplotlib inline
import networkx as nx

G = nx.DiGraph()
for i in range(6):
	   G.add_node(i+1)

G.add_edge(1,2)
G.add_edge(1,3)
G.add_edge(1,6)
G.add_edge(3,2)
G.add_edge(4,2)
G.add_edge(4,3)
G.add_edge(4,5)
G.add_edge(5,6)
G.add_edge(6,5)
G.add_edge(6,3)

A = nx.adjacency_matrix(G)
print(A.todense())




[[0 1 1 0 0 1]
 [0 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 1 0 1 0]
 [0 0 0 0 0 1]
 [0 0 1 0 1 0]]


In [31]:
import pandas as pd
url = "https://www.fun-mooc.fr/c4x/agrocampusouest/40001S03/asset/AnaDo_JeuDonnees_TemperatFrance.csv"
df = pd.read_csv(url, delimiter=";", encoding="latin1", index_col=0)
df

,Janv,Févr,Mars,Avri,Mai,Juin,juil,Août,Sept,Octo,Nove,Déce,Lati,Long,Moye,Ampl,Région
Bordeaux,5.6,6.6,10.3,12.8,15.8,19.3,20.9,21.0,18.6,13.8,9.1,6.2,44.50,-0.34,13.33,15.4,SO
Brest,6.1,5.8,7.8,9.2,11.6,14.4,15.6,16.0,14.7,12.0,9.0,7.0,48.24,-4.29,10.77,10.2,NO
Clermont,2.6,3.7,7.5,10.3,13.8,17.3,19.4,19.1,16.2,11.2,6.6,3.6,45.47,3.05,10.94,16.8,SE
Grenoble,1.5,3.2,7.7,10.6,14.5,17.8,20.1,19.5,16.7,11.4,6.5,2.3,45.10,5.43,10.98,18.6,SE
Lille,2.4,2.9,6.0,8.9,12.4,15.3,17.1,17.1,14.7,10.4,6.1,3.5,50.38,3.04,9.73,14.7,NE
Lyon,2.1,3.3,7.7,10.9,14.9,18.5,20.7,20.1,16.9,11.4,6.7,3.1,45.45,4.51,11.36,18.6,SE
Marseille,5.5,6.6,10.0,13.0,16.8,20.8,23.3,22.8,19.9,15.0,10.2,6.9,43.18,5.24,14.23,17.8,SE
Montpellier,5.6,6.7,9.9,12.8,16.2,20.1,22.7,22.3,19.3,14.6,10.0,6.5,43.36,3.53,13.89,17.1,SE
Nantes,5.0,5.3,8.4,10.8,13.9,17.2,18.8,18.6,16.4,12.2,8.2,5.5,47.13,-1.33,11.69,13.8,NO
Nice,7.5,8.5,10.8,13.3,16.7,20.1,22.7,22.5,20.3,16.0,11.5,8.2,43.42,7.15,14.84,15.2,SE


from this [post](https://stackoverflow.com/questions/14938541/how-to-improve-the-label-placement-for-matplotlib-scatter-chart-code-algorithm/15859652#15859652)

Networkx layout methods such as nx.spring_layout rescale the positions so that they all fit in a unit square (by default). Even the position of the fixed data_nodes are rescaled. So, to apply the pos to the original scatter_data, an unshifting and unscaling must be performed.

Note also that nx.spring_layout has a k parameter which controls the optimal distance between nodes. As k increases, so does the distance of the annotations from the data points.

In [39]:
import numpy as np
import networkx as nx

N = len(df.index)


def repel_labels( x, y, labels, k=0.01):
    G = nx.DiGraph()
    data_nodes = []
    init_pos = {}
    for xi, yi, label in zip(x, y, labels):
        data_str = 'data_{0}'.format(label)
        G.add_node(data_str)
        G.add_node(label)
        G.add_edge(label, data_str)
        data_nodes.append(data_str)
        init_pos[data_str] = (xi, yi)
        init_pos[label] = (xi, yi)

    pos = nx.spring_layout(G, pos=init_pos, fixed=data_nodes, k=k)

    # undo spring_layout's rescaling
    pos_after = np.vstack([pos[d] for d in data_nodes])
    pos_before = np.vstack([init_pos[d] for d in data_nodes])
    scale, shift_x = np.polyfit(pos_after[:,0], pos_before[:,0], 1)
    scale, shift_y = np.polyfit(pos_after[:,1], pos_before[:,1], 1)
    shift = np.array([shift_x, shift_y])
    for key, val in pos.items():
        pos[key] = (val*scale) + shift

    return pos

pos = repel_labels( df['Long'], df['Lati'], df.index, k=0.005)


In [40]:
import toyplot
canvas = toyplot.Canvas(width=600, height=300)
axes = canvas.cartesian(xlabel="Longitude", ylabel="Latitude")
for long, lati, ville in zip(df['Long'],df["Lati"],df.index):
    axes.text(pos[ville][0], pos[ville][1], ville, style={
        "text-anchor":"start","font-size":"14px"})
    axes.scatterplot(long, lati, color="black")



Bordeaux Brest Clermont Grenoble Lille Lyon Marseille Montpellier Nantes Nice Paris Rennes Strasbourg Toulouse Vichy -5 0 5 10 Longitude 42 45 48 51 Latitude